In [ ]:
import os
import sys
import importlib.util

sys.path.append('./src/')

from helpers.snakemake import nested_dict_update, get_git_revision_hash
from default_net_params import params as net_params
from data_preprocessing.cytoarchitecture import NeuronNumbers
from data_preprocessing.connectivity import SynapseNumbers
from network import Network

In [ ]:
# Set base path
base_path = os.getcwd()
experiments_path = os.path.join(base_path, 'experiments')

In [ ]:
# Load script from specified path sys.argv[1] (snakemake {input})
# Does nothing else than `import path/to/script as exp` would do
experiment_file = './experiments/experiment_stabilized_groundstate.py'
# experiment_file = './experiments/experiment_stabilized_bestfit.py'
conf_name, _ = os.path.splitext(os.path.basename(experiment_file))
conf_path = os.path.join(os.getcwd(), experiment_file)
spec = importlib.util.spec_from_file_location(conf_name, conf_path)
exp = importlib.util.module_from_spec(spec)
spec.loader.exec_module(exp)

In [ ]:
# Update default parameters with parameters specified in exp
nested_dict_update(net_params, exp.net_params)
outpath = net_params['outpath']

# Set simulation parameters
exp.sim_params['t_sim'] = 2000.0

# Set plotting parameters
exp.ana_params['plotRasterArea']['low'] = 1500
exp.ana_params['plotRasterArea']['high'] = 2000
exp.ana_params['plotRasterArea']['fraction'] = 0.5

# Set scaling parameters
scaling_factor = 0.005
net_params['N_scaling'] = scaling_factor
net_params['K_scaling'] = scaling_factor

### Creating the network

Here the network is created using the `NeuronNumbers` and `SynapseNumbers` classes. The `NeuronNumbers` class calculates the number of neurons in the network. The `SynapseNumbers` class calculates the number of synapses in the network and the synaptic weights.

In [ ]:
# Create Network class
NN = NeuronNumbers(
    surface_area=net_params['surface_area'],
    **net_params['cytoarchitecture_params']
)

# Create Synapse class
SN = SynapseNumbers(
    NN=NN,
    **net_params['predictive_connectomic_params']
)

In [ ]:

# Create Network
net = Network(NN, SN, net_params)

# Export the network
net.dump(outpath)

# Get the network hash
net_hash = net.getHash()

# To access net object key, use net.net.keys()
# Ex.: to access neuron numbers, use net.net['neuron_numbers']

### Running the simulation

In [ ]:
from default_sim_params import params as sim_params
from network import networkDictFromDump
from simulation import Simulation

In [ ]:
# Update default parameters with parameters specified in exp
nested_dict_update(net_params, exp.net_params)
nested_dict_update(sim_params, exp.sim_params)

In [ ]:
# Read network dict
net_folder = os.path.join(outpath, net_hash)
net_dict = networkDictFromDump(net_folder)

In [ ]:
# Create Simulation class, export it and calculate the hash
sim = Simulation(sim_params, net_dict)
sim.dump(net_folder)
sim_hash = sim.getHash()

In [ ]:
# Set output directory according to hashes, instantiate the Network
# and run the simulation
data_path = os.path.join(outpath, net_hash, sim_hash)
num_threads = int(4)
sim.setup(data_path, num_threads)
sim.simulate()

### Analysis

In [ ]:
from simulation import simulationDictFromDump
from analysis import Analysis
from default_ana_params import params as ana_params

In [ ]:
# Update default parameters with parameters specified in exp
nested_dict_update(net_params, exp.net_params)
nested_dict_update(ana_params, exp.ana_params)

# Read simulation dict
sim_folder = os.path.join(outpath, net_hash, sim_hash)
sim_dict = simulationDictFromDump(sim_folder)

# Create Analysis class and export it
ana = Analysis(ana_params, net_dict, sim_dict, sim_folder, base_path)
ana.dump(sim_folder)

# Do the analysis
# ana.fullAnalysis()

ana.popGids = ana._readPopGids()
ana.spikes = ana._readSpikes()
for area in net.net['area_list']:
    ana.plotRasterArea(area)